Si on souhaite installer un paquet sur le notebook

In [ ]:
import sys
!{sys.executable} -m pip install nompaquet

  Using cached https://files.pythonhosted.org/packages/5e/10/aa32dad071ce52b5502266b5c659451cfd6ffcbf14e6c8c4f16c0ff5aaab/Keras-2.2.4-py2.py3-none-any.whl
Could not install packages due to an EnvironmentError: [Errno 13] Permission non accordée: '/usr/lib/python3.7/site-packages/Keras-2.2.4.dist-info'
Consider using the `--user` option or check the permissions.



# Mercredi 6 Mars :
- On décide d'utiliser Keras comme libraire de ML car celle-ci rend le développement plus facile, en effet elle propose une API ergonomique qui vient se greffer sur plusieurs librairies au choix (Tensorflow, Pytorch ...)
- On décide d'utiliser Notebook pour suivre l'évolution du développement
- On décide d'utiliser Github pour partager le projet entre nous

# Mercredi 13 Mars :
- Mise en place de méthodes Agile
- Premier programme de ML capable de simuler une porte ET

In [ ]:
from keras.models import Sequential
from keras.layers import Dense
import numpy as np

model = Sequential()
model.add(Dense(16, activation='relu', input_dim=2))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['binary_accuracy'])

x = np.array([[0, 0], [0, 1], [1, 0], [1, 1]], "float32")
y = np.array([[0], [0], [0], [1]], "float32")

model.fit(x, y, epochs=200)

y_predict = model.predict(x).round()
print(y_predict)

# Mercredi 20 Mars :
- Premier réseau neuronal convolutif
- Collection de datasets : http://deeplearning.net/datasets/
- Datasets d'objets sous différents angles de vue :

	http://www.cs.columbia.edu/CAVE/software/softlib/coil-20.php

	http://www1.cs.columbia.edu/CAVE/software/softlib/coil-100.php

In [1]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
from keras.optimizers import SGD
from keras.datasets import mnist
from keras.utils import np_utils

(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train = x_train.reshape(x_train.shape[0], 1, 28, 28).astype('float32') / 255
x_test = x_test.reshape(x_test.shape[0], 1, 28, 28).astype('float32') / 255
y_train = np_utils.to_categorical(y_train)
y_test = np_utils.to_categorical(y_test)

model = Sequential()

model.add(Conv2D(32, (5, 5), input_shape=(1, 28, 28), activation="relu", data_format="channels_first"))
model.add(MaxPooling2D())
model.add(Conv2D(64, (5, 5), activation="relu"))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(1000, activation="relu"))
model.add(Dense(10, activation="softmax"))

model.compile(loss="categorical_crossentropy", optimizer=SGD(lr=0.01), metrics=['accuracy'])

model.fit(x_train, y_train, batch_size=200, epochs=10, validation_data=(x_test, y_test))

print(model.evaluate(x_test, y_test))

Using TensorFlow backend.


Instructions for updating:
Colocations handled automatically by placer.


Instructions for updating:
Use tf.cast instead.


Train on 60000 samples, validate on 10000 samples
Epoch 1/10


  200/60000 [..............................] - ETA: 4:51 - loss: 2.3228 - acc: 0.0950

...

 2800/60000 [>.............................] - ETA: 1:40 - loss: 2.2984 - acc: 0.1189

 3000/60000 [>.............................] - ETA: 1:38 - loss: 2.2981 - acc: 0.1217

 3200/60000 [>.............................] - ETA: 1:37 - loss: 2.2972 - acc: 0.1225

 3400/60000 [>.............................] - ETA: 1:36 - loss: 2.2964 - acc: 0.1241

KeyboardInterrupt: 